## Projekt IUM (zestaw 09, wariant 02)

Autorzy:
- Shpakovich Valeryia 317114
- Wolert Rafał 306020

### Spis treści:
1. Wybór modelu: bazowego i bardziej zaawansowanego modelu docelowego.
    - 1.1 Rozwiązanie problemu grupowania (pierwsze zadanie modelowania)
    - 1.2 Rozwiązanie problemu klasyfikacji wieloklasowej (drugie zadanie modelowania)
2. Implementacja aplikacji w formie mikroserwisu:
    - 2.1 Serwowanie predykcji przy pomocy danego modelu
    - 2.2 Realizacja eksperymentu A/B
3. Działanie implementacji
4. Spełnienie kryteriów biznesu/suckesu

### 1. Wybór modelu: bazowego i bardziej zaawansowanego modelu docelowego.

Nasz projekt zakładał dwa modele: jeden model związany z zadaniem clusteringu, a drugi związany z zadaniem klasyfikacji. Model clusteringu został zastosowany celem otrzymania danych wzbogaconych o gatunek dla danego utworu. Następnie na danych wzbogaconych o gatunek dla danego utworu (każdy utwór ma jeden gatunek utworu), została przeprowadzona klasyfikacja wieloklasowa.

Model klasyfikacji wieloklasowej jest modelem, który w naszym projekcie używany jest w środowisku produkcyjnym. Stąd model bazowy i bardziej zaawansowany dotyczy jedynie modelu klasyfikacji wieloklasowej.

#### 1.1 Rozwiązanie problemu grupowania

Proces budowy modelu dla problemu grupowania został przedstawiony w pliku: [nazwa_pliku].

Pierwsze zadanie modelowania było związane z odpowiednim przypisaniem gatunku dla danego utworu:
  - na zbiorze wszystkich piosenek stosujemy **clustering**, celem uzyskania grup, które posiadają cechy charakterystyczne dla każdego gatunku i są związane z rytmicznością, tanecznością, głośnością itd. danego utworu
  - wynikiem clusteringu są grupy, z których w każdej znajdują się piosenki z przypisanymi autorami. Artyści mają przypisane gatunki w których tworzą.
  - na podstawie gatunków, w których artyści tworzą, z pomocą heurystyk, jaki gatunek występuje w grupie najczęściej, określany będzie gatunek całej grupy.

Poniżej przedstawiono opisany proces:

  ![pierwszy_model](pierwszy_model.png)

Dzięki metodzie clusteringu otrzymujemy dane wzbogacone o gatunek dla danego utworu. 

[tu_napisac_o_procesie_budowania_modelu_i_o_wynikach]

#### 1.3 Rozwiązanie problemu klasyfikacji wieloklasowej (drugie zadanie modelowania)

Dane wzbogacone o gatunek dla danego utworu zostały użyte w procesie klasyfikacji wieloklasowej. Zmienna celu prezentowała się następująco:

- pop                     66271
- rock                    11758
- latin                    3343
- easy_listening            784
- jazz                      110
- traditional_folk           75
- r&b                        28
- acoustic                   15

Łącznie: 82384 rekordów.

Zastosowane cechy w klasyfikacji wieloklasowej to: ['danceability','key','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo'], gdzie zmienna celu to kolumna ['genre']. 

Wszystkie kolumny użyte w procesie klasyfikacji wieloklasowej, były kolumnami z wartościami numerycznymi, ale o innej skali - zastosowano skalowanie z użyciem StandardScaler() z biblioteki sklearn. Użyty algorytm w klasyfikacji to RandomForestClassifier. Zbiór testowy i treningowy został uzyskany w podziale zbioru danych według stosunku odpowiednio 20:80 (%).

Łącznie utworzono 4 modele klasyfikacji wieloklasowej, z której wybrano 2 jako modele A i B. W dwóch przypadkach modeli, użyto techniki SMOTE celem utworzenia syntetycznych danych i oversamplingu klas mniejszościowych, a co za tym idzie, uzyskania bardziej zbalansowanego zbioru danych.

Do strojenia modelu nr 2 i nr 4 użyto HalvingRandomSearchCV do poszukiwania optymalnego zbioru hiperparametrów: n_estimators, max_depth, min_samples_split oraz min_samples_leaf. 
Utworzono także scorer, na podstawie f1_score. Dla modelu 2 brano pod uwagę etykiety: ["easy_listening", "acoustic", "jazz", "latin", "r&b", "rock", "traditional_folk"], które były klasami mniejszościowymi, a dla modelu 4 (wszystkie dane były równe pod względem ilościowym), etykiety: ["latin", "pop", "rock"], które miały najmniejszy f1_score.

1. Model bez SMOTE i bez strojenia wyniki
   
 ![etap2_pierwszy_model](etap2_raport_grafika/bez_smote_niestrojony.png)



2. Model bez SMOTE i ze strojeniem wyniki
   
 ![etap2_drugi_model](etap2_raport_grafika/bez_smote_strojony.png)

W pierwszym przypadku zastosowano algorytm RandomForestClassifier z domyślnymi hiperparametrami. W drugim przypadku, zastosowano opisane wyżej strojenie hiperparametrów (trwało około 1.5h, ze zmniejszoną ilością przedziałów wartości parametrów do szukania, optymalny wyszukany zbiór to: RandomForestClassifier(max_depth=70, n_estimators=822)), które polepszyło kategorię: easy_listening z 0.26 na 0.27 pod względem f1-score, pogorszyło zaś w kategorii: latin z 0.33 na 0.31 oraz w kategorii traditional_folk z 0.44 na 0.25. 
W obu przypadkach bierzemy pod uwagę f1-score ze względu na niezbalansowany zbiór. Wyniki w obu modelach również wskazują na silne niezbalansowanie klasyfikacji gatunków, model generuje dużo błędnych predykcji i nie jest w stanie dobrze przyporządkowywać gatunków do klas, mamy wysoki f1-score tylko dla gatunku 'pop'.

3. Model z SMOTE i bez strojenia wyniki
   
 ![etap2_trzeci_model](etap2_raport_grafika/smote_niestrojony.png)

4. Model z SMOTE i ze strojeniem wyniki
   
 ![etap2_czwarty_model](etap2_raport_grafika/smote_strojony.png)


Celem większego zbalansowania klas, użyto techniki SMOTE do oversamplingu klas mniejszościowych. Widzimy, że w modelu nr 3, gdzie zastosowano technikę SMOTE, f1-score dla każdego gatunku jest bardzo wysoki, w niektórych przypadkach równy 1. Oznacza to, że możemy mieć do czynienia z nadmiernym dopasowaniem modelu. Strojenie hiperparametrów (około 3 godzinne), nie dało satysfakcjonujących wyników, bardzo pogorszyło jakość modelu. Strojenie hiperparametrów wskazało na dobór zbioru parametrów: (max_depth=80, n_estimators=2000), gdzie maksymalna liczba w parametrze n_estimators w zadanym strojeniu to 2000. Oznacza to, że gdybyśmy poszerzyli zakres parametru n_estimators, prawdopodobnie otrzymalibyśmy lepsze zbiór parametrów, natomiast było to zbyt czasochłonne.

Warto nadmienić, że optymalną techniką byłoby użycie techniki SMOTE zarówno pod względem oversamplingu klas mniejszościowych i undersamplingu klasy większościowej, bądź zastosowanie innej techniki undersamplingu w połączeniu z oversamplingiem SMOTE i odwrotnie. Prawdopodobnie wtedy, otrzymalibyśmy bardziej wiarygodne wyniki dla modelu.

Z tego powodu, jako modele A oraz B wybrano: model bez strojenia i bez SMOTE, model ze strojeniem i bez SMOTE (z powyższego opisu, model nr 1 i nr 2, oba znajdują się w folderze /modele

### 4. Spełnienie kryteriów biznesu/sukcesu

Określone kryteria sukcesów w ramach pierwszego etapu to:

Biznesowe:

Nowododanym wykonawcom skutecznie przypisywane są gatunki muzyczne.

Analityczne:

Po wstępnej analizie oraz zaproponowaniu modelu bazowego, będziemy starali się otrzymać przypisania gatunków mniej licznych do utworów. Obecnie, możemy zauważyć, że ze względu na niezbalansowany zbiór danych, mamy główne gatunki typu 'pop' czy 'rock'. Naszym kryterium sukcesu, opartym o model bazowym, będzie przypisanie gatunków mniej licznych, określonych jako gatunki główne, w większości utworów.

Zakładaliśmy także, że przypisanie gatunku do piosenki, bezpośrednio świadczy o gatunku, w jakim tworzy autor piosenki. 

Finalnie, możemy mówić o średniej skuteczności w przypisywaniu gatunków muzycznych nowododanym wykonawcom. Wybrany model z niską skutecznością przewiduje gatunek dla piosenki innej niż 'pop'. Głównym problemem w procesie naszych zadań modelowania okazał się niezbalansowany zbiór danych użyty do clusteringu. Zbiór ten, ze względu na użyte metody, nie został poprawiony i klasy nie zostały zbalansowane - wymagałoby to bardzo szczegółowego zbadania zbioru i określenia danych podgatunków do gatunków głównych (liczba podgatunków w niektórych przypadkach była równa nawet liczbie 200, a pomimo tego, klasa i tak nie była liczna. Średnio około 70 podgatunków należało do jednego gatunku) lub określenia innych gatunków głównych bardziej wpasowujących się w zbiór danych.
Zastosowanie niezbalansowanego zbioru wymagałoby także użycia bardziej złożonych technik niż technika SMOTE jedynie w celu oversamplingu klas mniejszościowych. Połączenie innych technik celem jednoczesnego oversamplingu klas mniejszościowych i undersamplingu klas większościowych, mogłoby doprowadzić do uzyskania lepszych określonych gatunków dla piosenek w procesie clusteringu, a potem w procesie klasyfikacji.


